In [2]:
# Import libraries, dependencies
import pandas as pd
import numpy as np
import requests
import json

# Other imports
from config import geoapify_key

In [3]:
# Import CSV and create a data frame
# Using a CSV file with all zip codes in Denver from:
# https://www.downloadexcelfiles.com/us_en/download-list-us-zip-codes-colorado-state#gsc.tab=0
# We selected only zip codes in Denver City using excel and saved  as denver_zipcodes.csv
file = 'resources/denver_zipcodes.csv'
denver = file_one_df = pd.read_csv(file)
denver_df = pd.DataFrame(denver)
denver_df

,Zip Code,City,County
0,80221,DENVER,ADAMS
1,80229,DENVER,ADAMS
2,80233,DENVER,ADAMS
3,80234,DENVER,ADAMS
4,80260,DENVER,ADAMS
5,80247,DENVER,ARAPAHOE
6,80201,DENVER,DENVER
7,80202,DENVER,DENVER
8,80203,DENVER,DENVER
9,80204,DENVER,DENVER


In [4]:
# Add columns for Population, Per Capita Income, Median Age
denver_df = denver_df.assign(Population = "")
denver_df = denver_df.assign(Income = "")
denver_df.rename(columns= {'Income': 'Per Capita Income'}, inplace=True)
denver_df =denver_df.assign(Age = "")
denver_df.rename(columns= {'Age': 'Median Age'}, inplace=True)
denver_df.head()

,Zip Code,City,County,Population,Per Capita Income,Median Age
0,80221,DENVER,ADAMS,,,
1,80229,DENVER,ADAMS,,,
2,80233,DENVER,ADAMS,,,
3,80234,DENVER,ADAMS,,,
4,80260,DENVER,ADAMS,,,


In [6]:
#Select parameters for request to geoapify, which will find all healthcare facilities in each zip code. (Kelsy)

# Set the geographical coordinates for Denver, CO
latitude = 39.742043
longitude = -104.991531

# Set the search parameters to capture desired zipcodes
radius = 32000
limit = 500
# offset = 500
filters = f"circle:{longitude},{latitude},{radius}"

# set up a parameters dictionary
params = {
    "categories": "healthcare",
    "filter":filters,
    "limit":limit,
    # "offset":offset,
    "apiKey":geoapify_key
}

# Set base URL
base_url = "https://api.geoapify.com/v2/places"

In [7]:
response = requests.get(base_url, params=params).json()

In [8]:
# Use a for loop to make an api request for a radius around Denver. Include each facility's name, zip code, subcategory(type), and wheelchair access status. (Kelsey)
# Empty list for data
health_facilities = []

# Retrieve the data with API request
print("Retrieving data...")
response = requests.get(base_url, params=params).json()

# Parse out facility name, categories, zipcode, lat, lng:
for feature in response["features"]:
    try:
        fac_name = feature["properties"]["name"]
    except (KeyError):
        fac_name = np.nan
    try:
        fac_type = feature["properties"]["categories"]
    except (KeyError):
        fac_type = np.nan
    try:
        fac_zip = feature["properties"]["postcode"]
    except (KeyError):
        fac_zip = np.nan
    try:
        fac_lat = feature["properties"]["lat"]
    except (KeyError):
        fac_lat = np.nan
    try:
        fac_lng = feature["properties"]["lon"]
    except (KeyError):
        fac_lng = np.nan

    # Append the data into health_facilities list
    health_facilities.append({"Facility Name": fac_name,
                              "Categories": fac_type,
                              "Zipcode": fac_zip,
                              "Lat": fac_lat,
                              "Lng": fac_lng})
    
    health_facilities

Retrieving data...


In [13]:
#Place the healthcare facility results into a DF(Kelsy).
health_facilities_df = pd.DataFrame(health_facilities)

pd.set_option("display.max_rows", None)
health_facilities_df

""


In [11]:
#Get population, median age, per capita income, of each zip code and add to data frame with healthcare facility info (Lisa)
#from https://data.colorado.gov/ (Lisa)
zip_codes = denver_df['Zip Code'].values.tolist()
zip_codes 

url_pop = 'https://data.colorado.gov/resource/ucnv-vw74.json?'
for index, row in denver_df.iterrows():
    #create the parameters, full url for the API call
    params_pop = {'zip_code': denver_df.loc[index, "Zip Code"],
              }
    response_pop = requests.get(url_pop, params=params_pop)
    response_pop = response_pop.json()

    # Store the zip's population, per capita income and median age in the denver_df DataFrame
    try:
        denver_df.loc[index, "Population"] = response_pop[0]["pop"]
        
    except (KeyError, IndexError):
        denver_df.loc[index, "Population"] = np.nan
    try:
        denver_df.loc[index, "Per Capita Income"] = response_pop[0]['per_cap_in']
    except (KeyError, IndexError):
        denver_df.loc[index, "Per Capita Income"] = np.nan
    try: 
        denver_df.loc[index, 'Median Age'] = response_pop[0]['med_age']
    except (KeyError, IndexError):
        denver_df.loc[index, "Median Age"] = np.nan
denver_df = denver_df.dropna()




In [12]:
denver_df

,Zip Code,City,County,Population,Per Capita Income,Median Age
0,80221,DENVER,ADAMS,40266,28618,34
1,80229,DENVER,ADAMS,56721,25054,32.1
2,80233,DENVER,ADAMS,50321,29967,33.1
3,80234,DENVER,ADAMS,29310,38436,35.2
4,80260,DENVER,ADAMS,36938,24013,30.1
5,80247,DENVER,ARAPAHOE,30161,32833,36
7,80202,DENVER,DENVER,15813,90952,34.4
8,80203,DENVER,DENVER,21368,52577,32.3
9,80204,DENVER,DENVER,34860,39623,32.6
10,80205,DENVER,DENVER,34884,44752,32.6


In [14]:
health_facilities_df.head()


""


In [ ]:
# Lisa
# Question 1: are healthcare facilities more dense where the population is dense? In other words: 

# Is there a correlation between pop of zip code and  number of facilities in the zip code?
# plot #of healthcare facilities per zip code against population per zip code (double bars with zip as x axis)


# TODO: scatter plot and linear regression of facility count vs population

In [ ]:
# Liz

# Question 2: Is there a correlation of per capita income with # of healthcare facilities by zip code in Denver?
# TODO: plot #of healthcare facilities per zip code against per cap income
# TODO: scatter plot and lin reg of  zip code against per cap income 

In [ ]:
# Kelsey
# Question 3: Is there a correlation of median age with # of healthcare facilities by zip code in Denver?
# TODO: plot #of healthcare facilities per zip code against median age
# TODO: scatter plot and lin reg of  zip code against median age

In [ ]:
# Bonus
# Quesion 4: What percentage of facilities are wheelchair accessible and what percentage by subcategory?
#TODO: count # and find % of facilities with wheelchair access
#TODO: If not very close to 100%, plot types of facility against wheelchair access(bar)
#TODO: map with color of facility dot by wheelchair access

In [ ]:
#TODO: written analysis (Liz first draft)

#TODO: prepare slides for presentation (Liz first draft)

#TODO: practice presentation(or at least be clear on respective roles)